In [2]:
# here we import all the libraries we will use in this session
import numpy as np  # numpy is a numerical library for Python
import itertools
from plotly import express as px
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, fixed
from checkers import checker_df, checker_lcoe, checker_parameters

# MJ2383: Lab 1 - Computing Levelised Cost of Electricity

In this lab, we are going to explore computation of an important metric called **levelised cost of electricity**, or LCOE for short.

The lab is broken up into several stages each of which culminates in writing a piece of Python code or doing some analysis. Don't worry if you've never written any Python before or done any programming - it is actually as easy as (or easier than) writing down equations.

in [stage 0](#Stage-0---Basic-Python-commands) we'll go through Python basics. If you've never used Python or programming before, you'll want to start here.

In [stage 1](#Stage-1---Compute-LCOE), we'll write a Python function which computes the levelised cost of electricity from input data. We'll then test that this function produces the result we expect. You'll be able to check that your function works at the end of the stage.

In [stage 2](#Stage-2---Explore-Parameter-Values), we'll then use the function you wrote to compute LCOE for one technology and explore what happens when you change the parameter values.

in [stage 3](#Stage-3:-Comparing-technologies), we'll use the LCOE function again to compare technologies against one another, and develop screen curves that allow us to visualise the trade offs between technologies as a function of load-factor.

## Housekeeping

- This is a Jupyter notebook. A notebook contains cells. Each cell contains either descriptive text (like this) or Python code.  You execute a cell by pressing `Shift + Enter`.  Or, you can use the `Run` button in the menu bar at the top of the page.

- Don't worry, you cannot break anything. If you think you've made a mistake, you can delete the content and try again. If you get really stuck, just reload the page and you'll be able to make a fresh start.

- Also, each stage allows you to begin afresh, so you can just move on when you are ready.

- If you like, work together in pairs sharing a screen.

- The notebook has been designed to help support your learning by enabling you to explore, but also to challenge. However, we are not trying to trick you. In most cases, the answers are in front of you.

- If you have questions, please ask us!
- If you have any suggestions for how this could be improved, or find a bug or problem, please submit an issue
  [here](https://github.com/KTH-dESA/MJ2383/issues/new/choose)

## Stage 0 - Basic Python commands
In this section, we will go through an overview of the most basics in python. If you already have knowledge of python you can skip this stage and move to [stage 1](#Stage-1---Compute-LCOE).

In Python we use the number symbol `#` to make a comment in the code. A comment is for clarification porpuses to the person reading the code, but it is ignored by the computer when you run the program, thus having no effect on your code output:

In [3]:
# this is a comment used to provide additional information

In Python there exist many types of data, in here we are going to cover the most basic ones: integers, floats, strings, lists and arrays.
```
5                        integer
2.6                      float
'this is a string'       str
[1, 2, 3, 4]             list
np.array([3, 4, 6, 4])   numpy.ndarray
```
we can use the function `type` to check the type of a value. Pass different data types to the `type` function to see the different output:

In [4]:
type(1) # change the data inside parenthesis

int

### Basic operators
You can actually use Python as a calculator to compute equations and store values into variables. This means that you can use basic mathematical operators:
```
+   addition
-   subtraction
*   multiplication
/   division
**  exponent
//  floor division
```

In [5]:
(3 + 5 * 10) / 4 - 10

3.25

In [6]:
3 ** 2

9

You can also store the output of an operation into a variable and us such variable for further operations:

In [7]:
x = 3 ** 2
y = 4 * x
print(x)
print(y)

9
36


### Lists and arrays
Lists are variables containing a sequence of ints, floats, strings etc., even other lists.... Whatever you want to put in your list. We create a list by using square brackets `[]` and passing the values of the list separated by commas.

In [8]:
my_list = [1, 2, 3, 4]
my_list

[1, 2, 3, 4]

An array is a list of values, which allows to make numerical calculations very efficiently. We create an array by calling the `numpy` library function `np.array()` and passing a list as argument (inside the parentesis):

In [9]:
my_array = np.array([1, 2, 3, 4])
my_array

array([1, 2, 3, 4])

Then we can use this array to make some elementwise computations or even pairwise computations between two arrays:

In [10]:
my_array * 2

array([2, 4, 6, 8])

In [11]:
my_array * np.array([4, 5, 6, 7])

array([ 4, 10, 18, 28])

We can even sum all of the values of the array by calling the function `sum()`

In [12]:
sum(my_array)

10

## Stage 1 - Compute LCOE

According to [OEE](http://www.open-electricity-economics.org/book/text/03.html) the formula to calculate LCOE is as follows:

\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y DF_y \cdot C_y}{\sum_{y=1}^Y DF_y \cdot G_y}\\)

where LCOE is the levelized cost in EUR per kWh, \\(C_{fix}\\) is the capital investment costs incurred for setting up the project, \\(C_y\\) are operational costs incurred in year \\(y\\), \\(Y\\) is the technical lifetime in years, and \\(G_y\\) is electricity generation in kWh. The costs are called levelized because they are “leveled” over all units of output. Levelized costs can be calculated for a specific power plant or for generic types of generation technologies.

Discount factor (DF) is calculated for each year \\(y\\) of the plant's technical lifetime

\\(DF_y =  (1 + r)^{-y}\\)

where r is the discount rate

Here we can store the technical lifetime of our technology in a *variable* called ``technical_lifetime``

In [13]:
technical_lifetime = 20

You can access the value stored in the *variable* at anytime, by typing its name and press `shift` + `enter`

In [14]:
technical_lifetime

20

We then create an *array* of years using the value stored in ``technical_lifetime``. 

An *array* is like a list of values. The command ``np.arange()`` takes a number as an argument and creates a *array* of the same length containing the values from 0 to one less than the number.

In [15]:
year = np.arange(technical_lifetime)
year

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

Now we calculate DF for each year in our plant's technical lifetime according to the formula we saw earlier:
\\(DF_y =  (1 + r)^{-y}\\); \\(r\\) is the discount rate, and \\(y\\) is the year.

**Implement the DF formula in the cell below. Remember that to exponentiate in Python we use** `**`.

**Q. What happens if you change the discount rate?**

In [16]:
discount_rate = 0.05
df = "replace me with the DF formula"
print(df) # we use the command print, to show the values stored inside the df variable

replace me with the DF formula


In [17]:
# Run this cell to check your answer
checker_df(df, discount_rate, year)

The argument `df` needs to be of type `np.array()`

Now we've calculated DF, let's get some data for the other parameters:

In [18]:
capital_costs = 1000.0  # €
operational_costs = np.repeat(500, technical_lifetime)  # €
electricity_generation = np.repeat(8760., technical_lifetime)  # kWh

The code above creates variables, each containing an *array* of values. We automatically generate the arrays using `np.repeat(50, technical_lifetime)`. This creates an array of length `technical_lifetime` where each element in the array contains the value `50`.

---

Remember the formula? Let's check it again:

\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y DF_y \cdot C_y}{\sum_{y=1}^Y DF_y \cdot G_y}\\)

Now, implement the LCOE formula in the cell below:

In [19]:
lcoe = "replace me with the LCOE formula"
print(lcoe)  # €/kWh

replace me with the LCOE formula


In [20]:
checker_lcoe(lcoe, capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime)

The argument `lcoe` needs to be of type `float`

At the moment, this seems a pretty arbitrary result. We're just putting in nonsense data, and we are getting nonsense in return. In stage 2, we'll start working with real data, but first, we need to make this calculation re-useable.

---

In Python, if we want to reuse a piece of code, we'll create a "function". To do this, you use the ``def`` command like this:

```python
def get_lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):
    # <<Something should go here>>
    return lcoe
```

- In the above example `lcoe` is the name of the function
- The list of names in the round brackets `()` are arguments
- Don't forget the `:` at the end of the list of arguments
- Use four spaces to indent your code
- The argument passed to `return` in the last line is what the function will give back when you run it

**Q. What should go in the function?**

Have a go below:

In [21]:
def get_lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):
    # Write your code next, be sure to include any variable needed to calculate the lcoe value.
    # Use the input variables of the function: 
    # (capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime)
    
    # <<Write your code here>>
    lcoe = 'replace me with some code'
    return lcoe

Test your `lcoe` function by running the cell below.

In [22]:
actual = get_lcoe(5000, np.array([100, 100, 100, 100]), np.array([8760, 8760, 8760, 8760]), 0.1, 4)
print(actual)
checker_lcoe(actual, 5000, np.array([100, 100, 100, 100]), np.array([8760, 8760, 8760, 8760]), 0.1, 4)

replace me with some code


The argument `lcoe` needs to be of type `float`

### Stage 1 - Summary

If you managed to get it to work, well done! You've written a Python function which computes LCOE. If not, don't worry, the answer is below! Make sure to write it in the function so the following steps of the Lab work.

- In this stage, we introduced the LCOE equation
\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y DF_y \cdot C_y}{\sum_{y=1}^Y DF_y \cdot G_y}\\)
and implemented it using a Python function.
- We also explored how the capital recovery factors weighs future years differently as a function of the discount rate.
- We learnt a lot of Python concepts including *variables*, *arrays*, and *arguments*.
- We learnt how to reuse Python code by writing a function.

In the next stage, we'll extend our function to make it more useful and investigate the LCOE calculation itself.

### Stage 1 - Answer LCOE Function

As you can see, we just need to fill in the gaps. First build the array of years, calculate the array of DF and then add the lcoe calculation.
```python
def get_lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):
    year = np.arange(technical_lifetime)
    df = (1 + discount_rate) ** - year
    lcoe = (capital_costs + sum(df * operational_costs)) / sum(df * electricity_generation)
    return lcoe
```

## Stage 2 - Explore Parameter Values

We'll now make use of an LCOE function to explore the effect of different parameter values for one technology - a combined cycle gas turbine. In the following implementation of LCOE, we make the assumption that the yearly values for operational costs and electricity generation are the same in each year of the technology's lifetime.

Here are the parameters for a combined cycle gas turbine (CCGT):

Parameter | Unit |Value
---|---|---
Lifetime| |25
Discount rate | |			8.0%
Station size | MW |	750.0
Overnight cost | €/kW | 750.0
Fixed O&M cost | €/kW	| 3.0
Efficiency | | 0.5
Fuel price | €/kWh| 0.03
Emission Factor	| ton/kW-yr| 1.8
Load factor | | 0.75

Here's the information we need to calculate the LCOE using the simple LCOE equation we developed earlier.

Parameter | Unit |Value
---|---|---
Capital cost | € | ?
Electricity generation | kWh | ?
Annual fixed O&M cost |	€	| ?
Annual variable O&M cost |€| ?
Annual operational cost | € | ?

**Q. What's the levelised cost of electricity for the CCGT plant?**
- work through the list of inputs methodically. 
- Pay close attention to the units.  
- Use Python variables to store information you reuse.

### Hints

- Remember there are approximately `365 * 24 = 8760` hours in a year.
- The load factor describes the proportion of those 8760 hours the plant is operational.
- Total operational costs are made up of fixed costs (not a function of activity) and variable costs (function of activity).
- The fixed costs are a function of capacity.
- The variable cost will be the product of the amount of fuel used and the fuel price.
- Fuel use can be computed from electricity generation and efficiency.
- Create any new variable if you need to.

In [23]:
# Add your working HERE

# 1. Populate some variables using the information from the table

lifetime = 25
discount_rate = 0.08
capacity = 750000 # kW
overnight_cost = 750.0 # €/kW
fixed_om_cost = 3 # €/kW
efficiency = 0.5
fuel_price = 0.03 # €/kWh
load_factor = 0.75

# 2. Calculate the intermediate values (you can use Python like a calculator)
# Use *+-/ for multiply, add, substract and divide.

capital_costs = capacity * overnight_cost # in €
# electricity_generation = 
# operational_costs = 

print(electricity_generation)
print(operational_costs)

[8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760.
 8760. 8760. 8760. 8760. 8760. 8760. 8760. 8760.]
[500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500]


In [24]:
# run this cell to check your answers   
checker_parameters(electricity_generation, operational_costs, capacity, load_factor, fixed_om_cost, fuel_price, efficiency)

Unexpected error: <class 'ValueError'> Make sure your electricity generation and operational values are of type float


In [25]:
# 3. Now use the earlier LCOE equation we developed to calculate LCOE
actual = get_lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, lifetime)
print(actual)

replace me with some code


---
Now we extend our calculation of LCOE with the parameters used above. We implement the same calculation you did in the `get_lcoe_input`. Then we use this function inside a new function called `extended_lcoe` to calculate the input parameters and the `lcoe` value using our previous definded `get_lcoe` function. You do not need to change anythiing here, just check them below and make sure to run both cells as we will need them in the next steps.

In [26]:
def get_lcoe_input(station_size, overnight_cost, fuel_efficiency, 
                   fuel_price, fixed_om_cost, load_factor, variable_om_cost):
    """Calculates the input parameters for the lcoe function
    
    Arguments
    ---------
    station_size : float
        The capacity of the technology in kW
    overnight_cost : float
        The capital cost of the technology in €/kW
    fuel_efficiency : float
        The ratio describing quantity of fuel required per unit of activity
    fuel_price : float
        The price paid per unit of input fuel in €/kWh
    fixed_om_cost : float
        The fixed operation and maintenance cost of the technology in €/kW
    load_factor : float
        The percentage of the year in which the technology generates electricity in %.
    variable_om_cost : float
        The variable operation and maintenance cost of the technology in €/kW
        
    Returns
    -------
    dict
        A dictionary containing:
            capital_cost : float 
                The capital cost of the technology €
            total_fixed_om_cost : float
                Fixed costs €
            annual_electricity_generation : float
                annual electricity production kWh
            total_variable_om_cost : float
                variable cost due to fuel use €
            annual_operational_cost : float
                annual operational cost €
            
    """
    HOURS_IN_YEAR = 8760
    
    capital_cost = station_size * overnight_cost
    total_fixed_om_cost = station_size * fixed_om_cost
    annual_electricity_generation = station_size * HOURS_IN_YEAR * load_factor

    total_variable_om_cost = ((annual_electricity_generation / fuel_efficiency) * fuel_price) + \
                             (annual_electricity_generation * variable_om_cost)
    annual_operational_cost = total_fixed_om_cost + total_variable_om_cost
    
    return {'capital_cost': capital_cost, 'total_fixed_om_cost': total_fixed_om_cost,
            'annual_electricity_generation': annual_electricity_generation, 
            'total_variable_om_cost': total_variable_om_cost, 'annual_operational_cost': annual_operational_cost}

In [27]:
def extended_lcoe(station_size, overnight_cost, fuel_efficiency, fuel_price, fixed_om_cost, load_factor, 
                  discount_rate, technical_lifetime, var_om_cost):
    """Calculates levelised cost of electricity as a function of useful parameters
    
    Arguments
    ---------
    station_size : float
        The capacity of the technology in kW
    overnight_cost : float
        The capital cost of the technology in €
    fuel_efficiency : float
        The ratio describing quantity of fuel required per unit of activity
    fuel_price : float
        The price paid per unit of input fuel in €/kWh
    fixed_om_cost : float
        The fixed operation and maintenance cost of the technology in €/kW
    load_factor : float
        The percentage of the year in which the technology generates electricity in %.
    discount_rate : float
        A decimal value less than 1
    technical_lifetime : int
        Technical lifetime of the technology in years
    var_om_cost : float
        Variable operational and maintenance cost
        
    Returns
    -------
    float
        The levelised cost of electricity in €/kWh
    """
    lcoe_params = get_lcoe_input(station_size, overnight_cost, fuel_efficiency, 
                                 fuel_price, fixed_om_cost, load_factor, var_om_cost)

    value = get_lcoe(lcoe_params['capital_cost'], lcoe_params['annual_operational_cost'], 
                 lcoe_params['annual_electricity_generation'], discount_rate, 
                 technical_lifetime)
    return value

So to perform the calculation again using our new LCOE function, we get:

In [28]:
ccgt_lcoe = extended_lcoe(capacity, overnight_cost, efficiency, fuel_price, fixed_om_cost, 0.75, 
                          discount_rate, lifetime, 0.003)
ccgt_lcoe

'replace me with some code'

### Q. What are the key parameters that determine the LCOE of the CCGT plant?

Run the next cell to create an interactive slider widget which allows you to play with the `extended_lcoe` function and answer the following questions:

1. Which is the most influential input parameter?
2. Are all the responses linear? Is it easy to tell?
3. What happens if gas is expensive?
4. If the price of the gas turbine doubles from €750/kW to €1500/kW, what's the effect upon LCOE at different load factors?
5. How important is discount rate compared to the fuel price?

In [29]:
interact(extended_lcoe, station_size=(100000,750000, 10000), overnight_cost=(300, 2000), fuel_efficiency=(0.3, 0.7, 0.01), fuel_price=(0.00  , 0.20, 0.01), fixed_om_cost=(1, 50, 0.001), load_factor=(0.01, 1.0, 0.01), 
         discount_rate=(0.01, 0.30, 0.01), technical_lifetime=(10, 60, 1), var_om_cost=(0.0, 0.011, 0.001))

interactive(children=(IntSlider(value=420000, description='station_size', max=750000, min=100000, step=10000),…

<function __main__.extended_lcoe(station_size, overnight_cost, fuel_efficiency, fuel_price, fixed_om_cost, load_factor, discount_rate, technical_lifetime, var_om_cost)>

### Stage 2 - Summary

- In this stage, we calculated the levelised cost for a CCGT plant.
- Then, we explored the LCOE equation interactively to try to understand what parameters influence the LCOE of a technology.

In the next stage, we compare LCOE across technologies.

# Stage 3: Comparing technologies

In the previous stages, we developed our understanding of the LCOE equation, and applied it to one technology. Now comes the *really* fun part, where we develop a comparison across different technologies.

We can now make a more advanced function which holds the logic to calculate LCOE when we have a number of extra input parameters. Now, we can pass a *list* (elements separated by commas inside square brackets) of parameter values, and the function returns a list of results.

You don't need to change anything in the following functions, just check them and run them.

In [36]:
def lcoe_params(station_sizes, overnight_costs, fixed_om_costs, load_factors, efficiencies, 
                 fuel_prices, discount_rates, technical_lifetimes, technology, var_om_costs):
    """Calculate LCOE for cartesian product of all parameter lists
    """
    results = []  # Create a list to hold the observations
    
    # Loop over cartesian product of parameter values
    for (capacity, capex, fixed_om_cost, load_factor, efficiency, 
        fuel_price, discount_rate, technical_lifetime, variable_om_cost) in itertools.product(
        station_sizes, overnight_costs, fixed_om_costs, load_factors, 
        efficiencies, fuel_prices, discount_rates, technical_lifetimes, var_om_costs):

        observation = {}  # Create a dictionary to store the observation
        
        lcoe_params = get_lcoe_input(capacity, capex, efficiency, 
                                     fuel_price, fixed_om_cost, load_factor, variable_om_cost)

        lcoe = extended_lcoe(capacity, capex, efficiency, fuel_price, fixed_om_cost, 
                             load_factor, discount_rate, technical_lifetime, variable_om_cost)
    
        observation['Technology'] = technology
        observation['StationSize'] = capacity
        observation['OvernightCost'] = capex
        observation['FixedOMCost'] = fixed_om_cost
        observation['LoadFactor'] = load_factor
        observation['Efficiency'] = efficiency
        observation['FuelPrice'] = fuel_price
        observation['DiscountRate'] = discount_rate
        observation['Lifetime'] = technical_lifetime
        observation['LCOE'] = lcoe
        
        year = np.arange(technical_lifetime)
        total_df = sum((1 + discount_rate) ** - year)
        
        observation['Fixed OM Cost'] = (total_df * lcoe_params['total_fixed_om_cost']) / \
                                       (total_df * lcoe_params['annual_electricity_generation'])
        observation['Variable OM Cost'] = (total_df * lcoe_params['total_variable_om_cost']) / \
                                          (total_df * lcoe_params['annual_electricity_generation'])
        observation['Capital Cost'] = lcoe_params['capital_cost'] / \
                                      (total_df * lcoe_params['annual_electricity_generation'])

        results.append(observation)
        
    return results

First, we'll use our new function to compare multiple technologies under their central operating characteristics, and explore how that changes under different discount rates. Run the folowing cell to pass all of the input values for each technology to the function. Give it a minute and you will se a plot below.

In [39]:
# Here we create an array of load factors from 0.1 to 1.0 in steps of 0.01
load_factors = np.arange(0.1, 1.0, 0.01)
# We set the discount rate to a fixed value for the moment
discount_rate = [0.05, 0.1] # %

# CCGT plant parameters
technical_lifetime = [25] # years
station_size = [750000]  # kW
fuel_price = [0.03] # €/kWh
overnight_cost = [750]  # €/kW
fixed_om_cost = [3.0] # €/kW
var_om_cost = [0.01]
fuel_efficiency = [0.5]

ccgt_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, 'CCGT', var_om_cost)

# Coal plants
technical_lifetime = [35] # years
station_size = [1000000] # kW
fuel_price = [0.01] # €/kWh
overnight_cost = [2000]  # €/kW
fixed_om_cost = [5.0] # €/kW
var_om_cost = [0.01]
fuel_efficiency = [0.4]

coal_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, 'Coal', var_om_cost)

# Nuclear plants
technical_lifetime = [50] # years
station_size = [1200000] # kW
fuel_price = [0.005] # €/kWh
overnight_cost = [4500]  # €/kW
fixed_om_cost = [2.0] # €/kW
var_om_cost = [0.01]
fuel_efficiency = [0.4]

nuclear_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, 'Nuclear', var_om_cost)

# Concentrating solar power
technical_lifetime = [25] # years
station_size = [500000] # kW
fuel_price = [0.0] # €/kWh
overnight_cost = [3785]  # €/kW
fixed_om_cost = [5.0] # €/kW
var_om_cost = [0.01]
fuel_efficiency = [1.0]

csp_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, 'CSP', var_om_cost)

# Wind turbines
technical_lifetime = [25] # years
station_size = [200000] # kW
fuel_price = [0.0] # €/kWh
overnight_cost = [1200]  # €/kW
fixed_om_cost = [2.0] # €/kW
var_om_cost = [0.0001]
fuel_efficiency = [1.0]

wind_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, 'Wind', var_om_cost)


plotting = pd.DataFrame(ccgt_results + coal_results + nuclear_results + csp_results + wind_results)
px.line(plotting, x='LoadFactor', y='LCOE', color='Technology', facet_col='DiscountRate',
        labels={'LoadFactor': 'LoadFactor (%)', 'LCOE': 'LCOE (€/kW)'})

- What do you see when you compare technologies at different load factors?
- What does the LCOE graph *not* show you?

Run the following cell to check another repreentation of the results:

In [40]:
df = plotting.melt(id_vars=['Technology', 'LoadFactor', 'DiscountRate'], value_vars=['Fixed OM Cost', 'Variable OM Cost', 'Capital Cost'])

px.area(df, x='LoadFactor', y='value', color='variable', facet_col='Technology', facet_row='DiscountRate',
        labels={'LoadFactor': 'LoadFactor (%)', 'value': 'LCOE (€/kW)'})

* What can you say about the different costs per technology?
* Does the wind technology really the have the lowest levelised cost of electricity?
* What happens if the cost of natural gas doubles? What happens if coal doubles in price?
* What happens if concrete gets cheaper?

That is it for today's lab, we hope you enjoyed it and that you got a good idea of what the LCOE is and how to use it.